In [1]:
cd shared

/srv/shared


/opt/tljh/user/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import string
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, f1_score
from tags import get_tags
import fasttext
from nltk import WordPunctTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset
from catboost import CatBoostClassifier
import tqdm

In [3]:
# !wget 'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.ru.zip' './'
# !unzip wiki.ru.zip

In [3]:
ft = fasttext.load_model(path='wiki.ru.bin')
wp_tokenizer = WordPunctTokenizer()

In [24]:
df = pd.read_pickle('./50k_25cats_FINAL.pkl')
X = df.text
y = df.topic

topic_to_number = {topic: i for i, topic in enumerate(y.unique())}
y = y.map(topic_to_number)

train_texts, test_texts, train_labels, test_labels = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
train_texts = train_texts[:4000]
test_texts = test_texts[:150]
train_labels = train_labels[:4000]
test_labels = test_labels[:150]

train_dataset = Dataset.from_pandas(pd.DataFrame({'text': train_texts, 'label': train_labels}))
test_dataset = Dataset.from_pandas(pd.DataFrame({'text': test_texts, 'label': test_labels}))

/tmp/ipykernel_36955/1148302802.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  train_texts = train_texts[:4000]
/tmp/ipykernel_36955/1148302802.py:10: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  test_texts = test_texts[:150]
/tmp/ipykernel_36955/1148302802.py:11: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the 

In [25]:
def rank_to_weight(rank, total_words, method='log'):
    if method == 'linear':
        return rank / total_words
    elif method == 'log':
        return np.log(rank+1) / np.log(total_words)
    else:
        raise ValueError("Invalid method. Choose 'linear', 'log', or 'sigmoid'.")

word_ranks = {word: rank + 1 for rank, word in enumerate(ft.get_words())}
total_words = len(word_ranks)

def get_phrase_embedding(phrase, word_ranks=word_ranks, total_words=total_words):
    """
    Convert phrase to a vector by aggregating its word embeddings with weighted average based on word ranks.
    """
    try:
        vector = np.zeros([ft.get_dimension()], dtype='float32')
        phrase = ''.join([char.lower() if char not in string.punctuation else ' ' for char in phrase])

        tokens = wp_tokenizer.tokenize(phrase)
        filtered_tokens = [word for word in tokens if word in ft.words]

        if filtered_tokens:
            word_vectors = []
            total_weight = 0
            for word in filtered_tokens:
                if word in word_ranks:
                    rank = word_ranks[word]
                    weight = rank_to_weight(rank, total_words)
                    word_vector = ft.get_word_vector(word) * weight
                    word_vectors.append(word_vector)
                    total_weight += weight

            if total_weight > 0:
                vector = sum(word_vectors) / total_weight
        return vector
    except Exception as e:
        print(f"Error occurred: {e}")
        return np.zeros([ft.get_dimension()], dtype='float32')

sample = 'исполняющая обязанности президента боливии жанин аньес рассказала что в ближайшие несколько дней будет выдан ордер на арест беглого эксглавы государства эво моралеса она заявила об этом местной газете los tiempos в частности моралеса решили арестовать за неуважение конституции боливии и ее переписывание по мнению аньес бывший президент должен ответить перед правосудием она рассказала что ходатайства уже поданы и добавила что действия моралеса должны расценивать как подстрекательство к мятежу и терроризму аньес также заявила что боливия нуждается в переменах она призвала моралеса соблюдать статус политического эмигранта и оставить страну в покое по ее словам новой власти удалось достигнуть примирения и необходимо провести выборы эво моралес объявил об отставке 10 ноября на фоне продолжающихся протестов после чего укрылся в мексике вместе с другими представителями руководства страны при этом он назвал произошедшее госпереворотом и заявил что готов вернуться в страну массовые беспорядки в боливии начались после объявления результатов выборов главы государства которые прошли 20 октября на них победу в первом туре одержал моралес его соперник экспрезидент карлос меса не признал итоги голосования власти обвинили в подтасовке результатов россия признала аньес временным президентом боливии в москве назвали события которые привели к отставке моралеса эквивалентными госперевороту однако посчитали происходящее внутренним делом страны'
vector = get_phrase_embedding(sample)
print(vector[:10])

[ 0.06081379  0.12583245  0.0389048  -0.2425245   0.05361604 -0.00301084
  0.04332336  0.04200802  0.10299081  0.02953421]


In [26]:
X_train = []
y_train = []

for row in tqdm.tqdm(train_dataset, desc="Building Training Data"):
    emb = get_phrase_embedding(row['text'])
    X_train.append(emb)
    y_train.append(row['label'])

X_train = np.array(X_train)
y_train = np.array(y_train)


Building Training Data:  70%|███████   | 2802/4000 [28:17<50:20,  2.52s/it]  

In [ ]:
model = CatBoostClassifier(iterations=1000, loss_function='MultiClass')
model.fit(X_train, y_train, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.079127
0:	learn: 3.1410112	total: 179ms	remaining: 2m 58s
1:	learn: 3.0698838	total: 354ms	remaining: 2m 56s
2:	learn: 2.9758472	total: 514ms	remaining: 2m 50s
3:	learn: 2.9076551	total: 675ms	remaining: 2m 47s
4:	learn: 2.8453407	total: 842ms	remaining: 2m 47s
5:	learn: 2.7749111	total: 1s	remaining: 2m 45s
6:	learn: 2.7103872	total: 1.16s	remaining: 2m 44s
7:	learn: 2.6422005	total: 1.32s	remaining: 2m 43s
8:	learn: 2.5750995	total: 1.49s	remaining: 2m 43s
9:	learn: 2.5100893	total: 1.66s	remaining: 2m 43s
10:	learn: 2.4583496	total: 1.82s	remaining: 2m 43s
11:	learn: 2.4029762	total: 1.99s	remaining: 2m 43s
12:	learn: 2.3526970	total: 2.16s	remaining: 2m 44s
13:	learn: 2.3095756	total: 2.33s	remaining: 2m 44s
14:	learn: 2.2717902	total: 2.5s	remaining: 2m 44s
15:	learn: 2.2262746	total: 2.67s	remaining: 2m 44s
16:	learn: 2.1815278	total: 2.84s	remaining: 2m 44s
17:	learn: 2.1417897	total: 3s	remaining: 2m 43s
18:	learn: 2.1033575	total: 3.17s	remaining: 2m 43s

In [ ]:
X_test = []
y_test = []

for row in tqdm.tqdm(test_dataset, desc="Building testing Data"):
    emb = get_phrase_embedding(row['text'])
    X_test.append(emb)
    y_test.append(row['label'])

X_test = np.array(X_test)
y_test = np.array(y_test)

Building testing Data: 100%|██████████| 150/150 [02:05<00:00,  1.20it/s]


In [ ]:
def evaluate(test_data):
    predicted_labels = [model.predict([emb])[0] for emb in X_test]
    weighted_f1 = f1_score(y_test, predicted_labels, average='weighted')
    print("Weighted F1 Score:", weighted_f1)

evaluate(test_dataset)

Weighted F1 Score: 0.732110626182314
